# Sharing code along with the data using CliMetLab


**Objective**:

The objective of this notebook is to illustrate how to create a climetlab dataset plugin for a dataset fomr these two file: `forecast_error.csv` and `soil_temperature.csv` (These files are located next to this notebook).

There are three main steps:

- Step 1: Create the plugin boilerplate structure using climetlab-plugin-tools.

- Step 2: Add your code to the plugin.

- Step 3: Push to github and publish to pypi (This is not covered here. See the general purpose tutorial about github and pipy)

## How to run this exercise

This exercise is in the form of a [Jupyter notebook](https://jupyter.org/). It can be "run" in a number of free cloud based environments (see three options below). These require no installation. When you click on one of the links below ([`Open in Colab`](https://colab.research.google.com/github/ecmwf-projects/mooc-machine-learning-weather-climate/blob/main/tier_2/data_handling/04-dataset-plugin.ipynb), [`Open in Kaggle`](https://kaggle.com/kernels/welcome?src=https://github.com/ecmwf-projects/mooc-machine-learning-weather-climate/blob/main/tier_2/data_handling/04-dataset-plugin.ipynb) or [`Launch in Deepnote`](https://deepnote.com/launch?url=https://github.com/ecmwf-projects/mooc-machine-learning-weather-climate/blob/main/tier_2/data_handling/04-dataset-plugin.ipynb)) you will be prompted to create a free account, after which you will see the same page you see here. You can run each block of code by selecting shift+control repeatedly, or by selecting the "play" icon. 

Advanced users may wish to run this exercise on their own computers by first installing [Python](https://www.python.org/downloads/), [Jupyter](https://jupyter.org/install) and [CliMetLab](https://climetlab.readthedocs.io/en/latest/installing.html).

<style>
td, th {
   border: 1px solid white;
   border-collapse: collapse;
}
</style>
<table align="left">
  <tr>
    <th>Run the tutorial via free cloud platforms: </th>
    <th><a href="https://colab.research.google.com/github/ecmwf-projects/mooc-machine-learning-weather-climate/blob/main/tier_2/data_handling/04-dataset-plugin.ipynb">
        <img src = "https://colab.research.google.com/assets/colab-badge.svg" alt = "Colab"></th>
    <th><a href="https://kaggle.com/kernels/welcome?src=https://github.com/ecmwf-projects/mooc-machine-learning-weather-climate/blob/main/tier_2/data_handling/04-dataset-plugin.ipynb">
        <img src = "https://kaggle.com/static/images/open-in-kaggle.svg" alt = "Kaggle"></th>
    <th><a href="https://deepnote.com/launch?url=https://github.com/ecmwf-projects/mooc-machine-learning-weather-climate/blob/main/tier_2/data_handling/04-dataset-plugin.ipynb">
        <img src = "https://deepnote.com/buttons/launch-in-deepnote-small.svg" alt = "Kaggle"></th>
  </tr>
</table>

Note that to use Kaggle, you need to enable an option on the notebook. Please follow the instructions here to do this https://stackoverflow.com/questions/68142524/cannot-access-internet-on-kaggle-notebook. 

## Let's begin the exercise...

In [ ]:
pip install climetlab --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
!ls *.csv

forecast_error.csv  soil_temperature.csv


#### Step 1: Create the plugin boilerplate structure using climetlab-plugin-tools.

In [ ]:
!climetlab help

Climetlab plugin tools are packaged separately, we need to install them as a plugin in order to have the shell command `climetlab create_plugin_dataset` available.

In [ ]:
!pip install climetlab-plugin-tools --quiet

In [ ]:
!climetlab help

Run from a shell terminal:

$ climetlab

(climetlab) create_plugin_dataset

Answer questions...

A new folder has now been created with all the code required to have a proper pip python package and have code included in a climetlab dataset plugin.

#### Step 2: Add your code to the plugin.

The previous created boilerplate code, let us now write some actual code to link it to the data.

Here is the file you want to edit.

In [ ]:
!ls climetlab-*/climetlab_*/*.py

In [ ]:
@normalize("parameter", ["tp", "t2m"])
def __init__(self, year, parameter):
    request = dict(parameter=parameter, url=URL, year=year)
    self.source = cml.load_source("url-pattern", PATTERN, request)

Let us edit this part and change it to:

In [ ]:
def __init__(self, parameter):
    self.source = cml.load_source("file", parameter + '.csv')
    # For a real plugin use "url" or "url-pattern" sources:
    # self.source = cml.load_source("url", URL_PREFIX + parameter + '.csv')
    # self.source = cml.load_source("url-pattern", PATTERN, {"parameter": parameter} )

The `pip` package need now to be installed. 

`pip install -e ./climetlab-my-plugin/. # adapt the name use the plugin name you choosed`

Note: using -e with pip
Warning for Jupyter users: you may need to restart your python notebook.


Let us test this. From a notebook or from ipython or from a python script:

In [ ]:
import climetlab as cml 
ds = cml.load_dataset('my-plugin', parameter = 'soil_temperature')
ds.to_pandas()


In [ ]:
ds = cml.load_dataset('my-plugin', parameter = 'forecast_error')
ds.to_pandas()

# Improving data usability
Data can be access as panda dataframe. Can we do better to help the end-user handling the data?

What about helping them fixing a typo?

In [ ]:
import climetlab as cml
cml.load_dataset('my-plugin', parameter = 'soiltemperature')

In [ ]:
# Add the climetlab decorator `@normalize`
@normalize("parameter", ['soil_temperature', 'forecast_error'])
def __init__(self, parameter):
    ...

# And retry previous cell.

This also take care of user using capitals:

In [ ]:
import climetlab as cml
ds = cml.load_dataset('my-plugin', parameter = 'SOIL_TEMPERATURE') # ok
ds = cml.load_dataset('my-plugin', parameter = 'Soil_Temperature') # ok

## Dates time parameters
Date and time as some ubiquitous in the climate and meteorology domains that we have developed specific tools to handle these input arguments.

Similar to `@normalize("parameter", ['soil_temperature', 'forecast_error'])`

Adding `@normalize("argument", "date(%Y-%m-%d)")` transforms the input as a string with the requested format.

Relevant CliMetLab documentation: https://climetlab.readthedocs.io/en/latest/contributing/normalize.html

# CliMetLab dataset plugin blueprint features

Here is a minimal example: https://github.com/ecmwf/climetlab-demo-dataset

Here is a real-life example: https://github.com/mchantry/climetlab-mltc-surface-observation-postprocessing

- Python pip package structure:
	- setup.py + MANIFEST
	- version file
- README
	- Links to notebook in colab/binder/etc. 
- Examples in notebooks:
	- Used in README links
	- Tested in github actions.
- Test in tests/*
	- Using pytest.
	- Used in github actions.
- Github actions: yaml files in .github/workflows/*.yml
	- Check code quality
	- Run tests (fron tests/*.py) on various platform and python versions
- Automated release of the pip package from github (need and account on pypi.org)
	- Make sure the tests pass.
	- Update the */version file
	- Trigger a release : https://docs.github.com/en/repositories/releasing-projects-on-github/managing-releases-in-a-repository
- Legal stuff: LICENCE/AUTHOR/CONTRIBUTORS


Compare to https://github.com/ecmwf/climetlab-demo-source/blob/master/climetlab_demo_source/__init__.py

Compare to https://github.com/ecmwf-lab/climetlab-google-drive-source/blob/main/climetlab_google_drive_source/__init__.py